In [1]:
import pandas as pd
from pickle import dump
pd.set_option("max_rows", None)

Read in race census data. Rename this file when you have a chance.

In [2]:
df_race = pd.read_csv('DECENNIALPL2020.P1_data_with_overlays_2021-10-04T202848.csv', error_bad_lines = False, header = None)

Sort out columns and index.

In [3]:
df_race.columns = df_race.iloc[1]

In [4]:
df_race.drop(df_race.index[1], inplace = True)

In [5]:
 df_race.drop(df_race.index[0], inplace = True)

Get rid of columns detailing multi-race breakdown but keep total of multi-race population.

In [6]:
df_race.drop(df_race.iloc[:, 11:73], axis = 1, inplace = True)

Get ride of punctuation.

In [7]:
df_race.columns = df_race.columns.str.strip().str.replace('[^\w\s]', '')

<ipython-input-7-3176a88a0be0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_race.columns = df_race.columns.str.strip().str.replace('[^\w\s]', '')


In [8]:
def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x
df_race = df_race.apply(remove_punctuation)

<ipython-input-8-5daa401cd4a4>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  x = x.str.replace('[^\w\s]','')


In [9]:
from unidecode import unidecode

df_race['County'] = df_race['County'].apply(unidecode)

KeyError: 'County'

Convert race/population info into floats.

In [ ]:
df_race[df_race.columns[2:]] = df_race[df_race.columns[2:]].astype(float)

Split county and state into two columns and drop old column.

In [ ]:
df_race[['County', 'State']] = df_race['Geographic Area Name'].str.split(',', expand=True)

In [ ]:
df_race.drop('Geographic Area Name', axis = 1, inplace = True)

Remove annoying words.

In [ ]:
df_race['County'] = df_race['County'].str.rstrip('County')

In [ ]:
df_race['County'] = df_race['County'].str.replace('(Borough|Area|Census|Municipality|and|City|Municipali)', '')

In [ ]:
df_race['County'] = df_race['County'].str.strip()

Chugach and Copper River counties in Alaska were combined between the time of the census and the time of the election. Here's my super duper annoying way of handling that...

In [ ]:
chugach = (df_race.loc[df_race['County'] == 'Chugach']) 

In [ ]:
copper_river = (df_race.loc[df_race['County'] == 'Copper River'])

In [ ]:
old_counties = chugach.append(copper_river)
valdez = pd.DataFrame(old_counties.sum(numeric_only = False, axis = 0)).T
df_race.drop(old_counties.index, axis = 0, inplace= True)
df_race = df_race.append(valdez)

In [ ]:
df_race = df_race.replace({'0500000US020630500000US02066': 'akcombined', 'ChugachCopper River': 'Valdez Cordova'})
df_race['State'] = df_race['State'].str.replace('Alaska Alaska', 'Alaska')

Drop Puerto Rico because they can not vote in presidential elections.

In [ ]:
pr = df_race.loc[df_race['State'].str.contains('Puerto Rico', case=False)]

In [ ]:
df_race = df_race.drop(pr.index, axis = 0)

Dropping Kalawoa. Only has 82 residents and does not come up in census pull.

In [ ]:
Kalawao = df_race.loc[df_race['County'].str.contains('Kalawao', case=False)]

In [ ]:
df_race = df_race.drop(Kalawao.index, axis = 0)

Rename annoying columns.

In [ ]:
df_race.rename(columns = {'Total': 'total_pop', 'TotalPopulation of one race': 'total_pop_one_race', 'TotalPopulation of one raceWhite alone':'pop_white', 'TotalPopulation of one raceBlack or African American alone': 'pop_african_american', 'TotalPopulation of one raceAmerican Indian and Alaska Native alone': 'pop_native', 'TotalPopulation of one raceAsian alone': 'pop_asian', 'TotalPopulation of one raceNative Hawaiian and Other Pacific Islander alone': 'pop_islander', 'TotalPopulation of one raceSome Other Race alone': 'pop_other', 'TotalPopulation of two or more races': 'total_pop_two_races'}, inplace = True)

Sort and reset index for joining.

In [ ]:
df_race = df_race.sort_values(by = ['State', 'County']).reset_index(drop = True)

In [ ]:
df_race.info()

In [ ]:
dump(df_race, open('df_race.pkl', 'wb'))